In [3]:
import pandas as pd
import numpy as np
import polars as pl
import pickle
import catboost

In [6]:
df_train = pd.read_csv("/root/kag_comp/UM_MCTS/Data/train.csv")
df_test = pd.read_csv("/root/kag_comp/UM_MCTS/Data/test.csv")
df_concepts = pd.read_csv("/root/kag_comp/UM_MCTS/Data/concepts.csv")

In [7]:
def checking_set(col, start, end) :
    """
    포함하는 계층형 구조를 지닌 행들을 식별할 때 사용
    """
    lst = list(df_train.columns)[start:end]
    if (df_train.loc[df_train[col] == 1,].iloc[:, start:end] == pd.Series({c:0 for c in lst})).all(axis = 1).sum() == 0 :
        print("X = 1 : 하나라도 1인 게 있음(문제 없음)")
    else :
        print("X = 1 : 전부 0인 게 있음(문제 있음)")

    if (df_train.loc[df_train[col] == 0,].iloc[:, start:end] == pd.Series({c:1 for c in lst})).sum().sum() == 0 :
        print("X = 0 : 전부 0임(문제 없음)")
    else :
        print("X = 0 : 하나라도 1인게 있음(문제 있음)")

def frequency_is_in(colindex) :
    if (df_train.iloc[:, colindex+1].loc[df_train.iloc[:, colindex] == 1] == 0.0).sum() != 0 :
        print("상위 노드에서 1일 때, 0인 경우가 있음")
    else :
        print("이상 없음")

    if (df_train.iloc[:, colindex+1].loc[df_train.iloc[:, colindex] == 0] != 0.0).sum() != 0 :
        print("상위 노드에서 0일 때, 0이 아닌 경우가 있음")
    else :
        print("이상 없음")

In [8]:
lst = []

for i, l in enumerate(df_train) :
    lst.append(df_train[l].unique())

In [9]:
lst2 = []

for i, l in enumerate(df_train) :
    lst2.append(list(df_train[[l]].groupby(l).agg({l : 'count'})[l]))

In [10]:
set(df_concepts.Name) - set(df_train.columns)

{'SkillTrace',
 'SkillTraceErrorIntercept',
 'SkillTraceErrorSlope',
 'SkillTraceScore',
 'SkillTraceTrials'}

**피쳐 설명이 엇갈리는 부분이 있으므로, (https://ludii.games/searchConcepts.php) 를 참고할 것**

In [11]:
df_train.head()

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333


In [17]:
df_train.columns

Index(['Id', 'GameRulesetName', 'agent1', 'agent2', 'Properties', 'Format',
       'Time', 'Discrete', 'Realtime', 'Turns',
       ...
       'DoLudeme', 'Trigger', 'PlayoutsPerSecond', 'MovesPerSecond',
       'EnglishRules', 'LudRules', 'num_wins_agent1', 'num_draws_agent1',
       'num_losses_agent1', 'utility_agent1'],
      dtype='object', length=814)

In [67]:
with open('trained_model_for_feature_importances.pkl', 'rb') as f :
    predictr = pickle.load(f)
    
importances = {i:j for i, j in zip(predictr.feature_names_, predictr.get_feature_importance())}
df_features = pd.DataFrame({'uniqueset' : lst, 'colname' : df_train.columns, 'description' : [' ', ' ', ' ', ' '] + list(df_concepts.iloc[:804,].Description) + [' ',' ',' ', ' ', ' ', ' '], 'LeafNode' : [' ', ' ', ' ', ' '] + list(df_concepts.iloc[:804,].LeafNode) + [' ',' ',' ', ' ', ' ', ' '], 'unique_count' : lst2})\
.assign(importance = lambda _df : _df.colname.map(lambda x : importances[x] if x in importances else pd.NA))

**특별한 표기가 없다면 반응변수는 전부 바이너리 변수**

Stochastic : 게임에 확률적 요소가 포함됩니다.

Asymmetric : 규칙 및 파워의 비대칭성이 존재합니다.

> AsymmetricForces : 플레이어의 파워가 다릅니다.
>
> > AsymmetricPiecesType : 각 플레이어가 보유한 말의 종류가 다릅니다. (리프 노드)

PlayersWithDirections : 플레이어들이 방향을 사용합니다.

Cooperation : 플레이어들은 협력해야 합니다.

> **관측수가 적고 의미가 적어보이는 리프 노드**
>
> Team : 게임에는 플레이어 팀이 포함됩니다.

(38) Shape : 게임판의 모양이 존재합니다.

> **관측수가 적고 의미가 적어보이는 리프 노드**
>
> SpiralShape : 나선형 보드입니다.
>
> StarShape : 별 모양 보드입니다.

(54) Tiling : 보드는 타일로 이뤄집니다.

> **관측수가 적고 의미가 적어보이는 리프 노드**
>
> SpiralTiling : 나선형 타일입니다.

(67) MancalaBoard : 만칼라 형태의 게임이다. **(하위 노드의 값이 전부 0인 행이 존재한다.)**

(74) AlquerqueBoard : 앨커키 형태의 게임이다. **(하위 노드의 값이 전부 0인 행이 존재한다.)**

(79) ThreeMensMorrisBoard : 쓰리 맨즈 모리스 보드이다.

(87) Track : 보드에 트랙이 존재한다. **(하위 노드의 값이 전부 0인 행이 존재한다.)**

(93) PlayableSites : 플레이 가능한 구역

> (97) NumPlayableSitesOnBoard : 보드 위에 사용가능한 위치의 수. 여기부터 수치형이 존재한다.

(98) NumColumns : 보드는 n개의 열로 구성되어 있다. ~ (120) NumPhasesBoard : 보드 변화 페이즈

(121) Hand : 플레이어가 손에 기물을 가질 수 있다.

(122) NumContainers : 저장공간은 n개가 존재한다.

(123) NumPlayableSites : 게임에서 전체 사용 가능한 장소가 n개 존재한다. (제거 가능)

(125) Piece : 게임에서 말을 사용합니다.

(126) PieceValue : 말이 값을 가집니다.

(127) PieceRotation : 말이 회전할 수 있습니다.

(128) PieceDirection : 말이 이동방향을 가집니다.

(129) Dice : 주사위를 사용합니다.

> DiceD2 : 동전을 사용합니다.
>
> DiceD4 : 4면체 주사위가 사용됩니다.
>
> DiceD6 : 6면체 주사위가 사용됩니다.

(137) LargePiece : 게임에서 큰 말을 사용합니다.

(138) Tile : 게임에서 타일을 사용합니다.

(139) NumComponentsType : 게임에 총 n개의 구성요소가 포함됩니다.

(140) NumComponentsTypePerPlayer : 각 플레이어는 n개의 구성요소를 소유합니다. - CV 필요, 사실상 139번과 같은 변수

(141) NumDice : 주사위의 개수

(143) Meta : 특정 조건에서 최우선시되는 전역적 규칙이 존재합니다.

> SwapOption : 두 번째 플레이어가 색상을 바꿀 수 있는 선택지를 가집니다.
>
> TurnKo : 단일 턴 내에서 동일한 말의 위치를 반복할 수 없습니다.
>
> PositionalSuperko : 게임 전체에서 동일한 말의 위치가 반복될 수 없습니다.
>
> AutoMove : 특정 위치와 관련된 모든 움직임을 자동으로 적용합니다.

(152) Start : 시작 규칙

> PiecesPlacedOnBoard : 게임 시작 시 보드 위에 일부 말을 배치합니다.
>
> PiecesPlacedOutsideBoard : 게임 시작 시 보드 외부에 일부 말을 배치합니다.
>
> InitialRandomPlacement : 게임 시작 시 일부 말을 무작위로 배치합니다.
>
> InitialScore : 게임 시작 시 초기 점수를 세팅합니다.
>
> InitialCost : 게임 시작 시 계량 요소에 초기 비용을 세팅합니다.
>
> NumStartComponentsBoard : 시작 시 보드 위에 놓인 개체의 총 수
>
> NumStartComponentsHand : 시작 시 플레이어가 소지한 개체의 총 수

(169) NoSiteMoves : 플레이어가 특정 위치와 관련 없는 이동을 합니다.

> VoteDecision : 플레이어가 투표를 진행합니다.
>
> SwapPlayersDecision : 플레이어가 순서를 교환할 수 있습니다.
>
> > SwapPlayersDecisionFrequency : 순서 교환 발생의 빈도
>
> PassDecision : 플레이어가 턴을 넘길 수 있습니다.
>
> > PassDecisionFrequency : 턴 패스 발생의 빈도

(182) SingleSiteMoves : 특정 위치(말이나 기물 등에 대하여)에서 수행되는 움직임을 합니다.

(191) TwoSitesMoves : 두 개의 지점(출발, 목표 등)을 대상으로 수행되는 움직임을 합니다.

> (192) StepDecision : 한 지점에서 다른 지점으로 이동을 결정
>
> (200) SlideDecision : 한 지점에서 다른 지점으로 미끄러지듯 이동을 결정
>
> (208) LeapDecision : 한 지점에서 다른 지점으로 뛰어넘는 이동을 결정
>
> (216) HopDecision : 한 지점에서 다른 지점으로 말들을 연속적으로 뛰어넘는 이동 결정 -> CV가 필요, 자료가 이상함.
>
> (232) FromToDecision : 한 지점에서 다른 지점으로 말을 이동(???)

(248) MovesNonDecision : 결정과 상관없는 이동을 합니다.

> (249) MovesEffects : 이동의 효과가 존재합니다.
>
> > (266) Sow : 돌이나 씨앗 등을 보드의 특정 위치에 배치합니다.
> >
> > (288) SetMove : 게임 내 상태를 설정하거나 조작하는 움직임입니다.
> () MovesOperators : 

In [220]:
df_features.iloc[169:200, ]

,uniqueset,colname,description,LeafNode,unique_count,importance
169,"[0, 1]",NoSiteMoves,Frequency of Rotation Decision move.,1,"[189300, 43934]",0.011842
170,[0],BetDecision,Moves.,0,[233234],<NA>
171,[0],BetDecisionFrequency,Decide to step.,0,[233234],<NA>
172,"[0, 1]",VoteDecision,Frequency of Step Decision move.,1,"[231664, 1570]",0.0
173,[0],VoteDecisionFrequency,Decide to step to an empty site.,0,[233234],<NA>
174,"[0, 1]",SwapPlayersDecision,Frequency of Step Decision To Empty move.,1,"[227532, 5702]",0.010928
175,"[0.0, 0.01]",SwapPlayersDecisionFrequency,Decide to step to a friend piece.,0,"[232860, 374]",0.086083
176,[0],ChooseTrumpSuitDecision,Frequency of Step Decision To Friend move.,1,[233234],<NA>
177,[0],ChooseTrumpSuitDecisionFrequency,Decide to step to an enemy piece.,0,[233234],<NA>
178,"[0, 1]",PassDecision,Frequency of Step Decision To Enemy move.,1,"[196714, 36520]",0.007159


In [4]:
## 애매한 변수들
NeedToCV = [172, 174, 175, 178, 217, 218, 247, 254, 289, 291] # 애매하면 그냥 넣고 CV하는 게 좋음.
Piuungsin = [303, 304] ## 얘넨 뭐냐 그냥

## categorical로 하면 좋을 것 같은 변수들
PersudoCat = [141, 181] ## 주사위의 개수 / 턴 패스 발생의 빈도

## 해당 행동의 존재 여부
crossedBinary = [174, 180, 183, 185, 187, 189, 194, 196, 198, 202, 204, 206, 210, 212, 214,] ## 바이너리, 안쓰는게 좋을듯
## > 이거 쓰는 게 좋을 것 같은게, 1임에도 빈도가 0인 상황이면 그것도 의미가 있는 거지...

## 이상한 그룹 HopDecision
HopDecision_binary = [218, 220, 222, 228, 230] ## 얘는 왠만해선 안쓰는 게 좋겠고... 아마도
HopDecision_freq = [219, 221, 223, 227, 229, 231] ## 얘는 쓰고...
HopDecision_root = [216, 217] ## 얘는 CV

## 진짜 이상한 그룹 FromToDecision
FromToDecision_binary = [234, 236, 238, 240, 242] ## 안쓰는 게 좋겠지...?
FromToDecision_freq = [235, 237, 239, 241, 243] ## 얘는 일단 씀
FromToDecision_root = [232, 233] ## 얘는 왠만해선 쓰고, CV

## Moves Effect
MovesEffects_binary = [252, 254, 258, 266, 268, 280, 282, 286, 293, 295, ] ## 얘들은 넣는 게 더 좋을 수도 있음

In [3]:
[12, 21, 28, 30, 39, 40, 41, 42, 43, 44, 45, 47, 49, 51, 52,
 55, 56, 57, 59, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
 88, 89, 91, 92, 94, 95, 96] + list(range(97, 121)) + [121,
 122, 123, 125, 126, 127, 128, 129, 130, 132, 133, 137, 138,
 139, 141, 143, 145, 147, 149, 151, 153, 154, 155, 156, 159,
 160, 161, 169, 172, 175, 178, 179, 181, 184, 186, 188, 190,
 195, 197, 199, 203, 205, 207, 211, 213, 215] + crossedBinary + HopDecision_freq + FromToDecision_freq + [
 245, 247, 252, 261, 263, 265, 267, 270, 272, 274, 276, 277,
 278, 279, 281, 283, 285, 287, 290, 292, 294, 296, 305, 307,
 309, 311, 313, 320, 321, 322, 323
 ] + MovesEffects_binary

In [24]:
# print(df_train.utility_agent1.mean())
# print(df_train.loc[df_train.AlquerqueBoardWithEightTriangles == 1,].utility_agent1.mean())
checking_set('SingleSiteMoves', 183, 191)
# 0 in df_train.NumPlayableSites.unique()

X = 1 : 하나라도 1인 게 있음(문제 없음)
X = 0 : 하나라도 1인게 있음(문제 있음)


In [20]:
df_train.shape

(233234, 814)

In [58]:
df_train.loc[df_train.SwapPiecesDecision == 0, "SwapPiecesDecisionFrequency"].unique()

array([0.])

In [116]:
df_features.loc[df_features.importance.isna() == False].loc[(lambda _df : _df.importance != 0.0)].importance.astype('float').describe()

count    350.000000
mean       0.285714
std        1.612332
min        0.000145
25%        0.009770
50%        0.028400
75%        0.095125
max       24.878423
Name: importance, dtype: float64

In [82]:
print(df_features.importance.mean(), df_features.importance.median())

0.1347708894878706 0.0


In [174]:
(df_train.iloc[:, [289+2*i for i in range(8)]].sum(axis = 1).map(lambda x : x if x == 0 else 1) != df_train.SetMove).sum()

0

In [212]:
(df_train.StepEffectFrequency.loc[df_train.StepEffect == 0] != 0.0).sum()

0

In [180]:
(df_train.SetNextPlayer == 0).sum()

232676

In [179]:
(df_train.SetNextPlayerFrequency == 0).sum()

131600

In [219]:
frequency_is_in(307)

상위 노드에서 1일 때, 0인 경우가 있음
이상 없음


In [225]:
checking_set("MovesOperators", 320, 324)

X = 1 : 하나라도 1인 게 있음(문제 없음)
X = 0 : 전부 0임(문제 없음)


In [226]:
df_features.iloc[324:350, ]

,uniqueset,colname,description,LeafNode,unique_count,importance
324,"[0, 1]",Capture,Space conditions.,0,"[147888, 85346]",0.0
325,"[0, 1]",ReplacementCapture,Line Detection.,1,"[196748, 36486]",0.0
326,"[0.0, 0.21, 0.08, 0.02, 0.05, 0.06, 0.01, 0.16...",ReplacementCaptureFrequency,Connected regions detection.,1,"[202762, 1854, 4092, 1856, 3018, 1308, 2286, 4...",0.026462
327,"[0, 1]",HopCapture,Detect a group.,1,"[198456, 34778]",0.016341
328,"[0.0, 0.01, 0.04, 0.08, 0.11, 0.13, 0.35, 0.09...",HopCaptureFrequency,Detect if a site is in a region.,1,"[200914, 904, 550, 2202, 2728, 3454, 2570, 171...",0.918508
329,"[0, 1]",HopCaptureMoreThanOne,Loop detection.,1,"[229386, 3848]",0.006378
330,"[0.0, 0.03, 0.06, 0.01, 0.04, 0.08, 0.11, 0.13...",HopCaptureMoreThanOneFrequency,Pattern detection.,1,"[191040, 4088, 724, 2718, 3184, 3454, 3114, 27...",0.238434
331,"[0, 1]",DirectionCapture,Path extent detection.,1,"[232856, 378]",0.003409
332,"[0.0, 0.55, 0.54]",DirectionCaptureFrequency,Territory detection.,1,"[232856, 184, 194]",0.0
333,"[0, 1]",EncloseCapture,Check region filled by pieces.,1,"[231046, 2188]",0.018764


> 그냥 이정도까지만 하고 CV해서 변수선택 할까?

In [29]:
## Cross Validation
from sklearn.model_selection import StratifiedKFold